# Topic Modeling (Using gensim)


In [5]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

from pprint import pprint

import spacy

import pickle
import re
import pyLDAvis
import pyLDAvis.gensim

import matplotlib.pyplot as plt
import pandas as pd
import nltk
import numpy as np

## Data Preparation

In [6]:
corpus = [
    'The sky is blue and beautiful.', 'Love this blue and beautiful sky!',
    'The quick brown fox jumps over the lazy dog.',
    "A king's breakfast has sausages, ham, bacon, eggs, toast and beans",
    'I love green eggs, ham, sausages and bacon!',
    'The brown fox is quick and the blue dog is lazy!',
    'The sky is very blue and the sky is very beautiful today',
    'The dog is lazy but the brown fox is quick!'
]
labels = [
    'weather', 'weather', 'animals', 'food', 'food', 'animals', 'weather',
    'animals'
]

wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')


def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I | re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokeanize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc


normalize_corpus = np.vectorize(normalize_document)
norm_corpus = normalize_corpus(corpus)
norm_corpus

array(['sky blue beautiful', 'love blue beautiful sky',
       'quick brown fox jumps lazy dog',
       'kings breakfast sausages ham bacon eggs toast beans',
       'love green eggs ham sausages bacon',
       'brown fox quick blue dog lazy', 'sky blue sky beautiful today',
       'dog lazy brown fox quick'], dtype='<U51')

In [7]:
norm_corpus_tokens = [text.split(" ") for text in norm_corpus]
id2word = Dictionary(norm_corpus_tokens)
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in norm_corpus_tokens]
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1)]]


In [9]:
[[(id2word[i], freq) for i, freq in doc] for doc in corpus[:1]]

[[('beautiful', 1), ('blue', 1), ('sky', 1)],
 [('beautiful', 1), ('blue', 1), ('sky', 1), ('love', 1)],
 [('brown', 1),
  ('dog', 1),
  ('fox', 1),
  ('jumps', 1),
  ('lazy', 1),
  ('quick', 1)]]

## LDA Model Building

In [10]:
# Build LDA model
lda_model = LdaModel(corpus=corpus,
                   id2word=id2word,
                   num_topics=10,
                   random_state=0,
                   chunksize=100,
                   alpha='auto',
                   per_word_topics=True)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.058*"ham" + 0.058*"sausages" + 0.058*"bacon" + 0.058*"eggs" + '
  '0.058*"toast" + 0.058*"fox" + 0.058*"lazy" + 0.058*"dog" + 0.058*"brown" + '
  '0.058*"quick"'),
 (1,
  '0.050*"blue" + 0.050*"beautiful" + 0.050*"sky" + 0.050*"quick" + '
  '0.050*"brown" + 0.050*"dog" + 0.050*"lazy" + 0.050*"fox" + 0.050*"love" + '
  '0.050*"eggs"'),
 (2,
  '0.050*"sky" + 0.050*"blue" + 0.050*"beautiful" + 0.050*"lazy" + 0.050*"dog" '
  '+ 0.050*"fox" + 0.050*"quick" + 0.050*"brown" + 0.050*"love" + '
  '0.050*"eggs"'),
 (3,
  '0.124*"dog" + 0.124*"quick" + 0.124*"fox" + 0.124*"brown" + 0.124*"blue" + '
  '0.124*"lazy" + 0.065*"beautiful" + 0.065*"love" + 0.065*"sky" + '
  '0.006*"sausages"'),
 (4,
  '0.050*"blue" + 0.050*"sky" + 0.050*"quick" + 0.050*"beautiful" + '
  '0.050*"love" + 0.050*"brown" + 0.050*"lazy" + 0.050*"dog" + 0.050*"fox" + '
  '0.050*"ham"'),
 (5,
  '0.050*"sky" + 0.050*"blue" + 0.050*"beautiful" + 0.050*"lazy" + 0.050*"dog" '
  '+ 0.050*"quick" + 0.050*"brown" + 0.050*"f

In [11]:
# Print the Keyword
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.058*"ham" + 0.058*"sausages" + 0.058*"bacon" + 0.058*"eggs" + '
  '0.058*"toast" + 0.058*"fox" + 0.058*"lazy" + 0.058*"dog" + 0.058*"brown" + '
  '0.058*"quick"'),
 (1,
  '0.050*"blue" + 0.050*"beautiful" + 0.050*"sky" + 0.050*"quick" + '
  '0.050*"brown" + 0.050*"dog" + 0.050*"lazy" + 0.050*"fox" + 0.050*"love" + '
  '0.050*"eggs"'),
 (2,
  '0.050*"sky" + 0.050*"blue" + 0.050*"beautiful" + 0.050*"lazy" + 0.050*"dog" '
  '+ 0.050*"fox" + 0.050*"quick" + 0.050*"brown" + 0.050*"love" + '
  '0.050*"eggs"'),
 (3,
  '0.124*"dog" + 0.124*"quick" + 0.124*"fox" + 0.124*"brown" + 0.124*"blue" + '
  '0.124*"lazy" + 0.065*"beautiful" + 0.065*"love" + 0.065*"sky" + '
  '0.006*"sausages"'),
 (4,
  '0.050*"blue" + 0.050*"sky" + 0.050*"quick" + 0.050*"beautiful" + '
  '0.050*"love" + 0.050*"brown" + 0.050*"lazy" + 0.050*"dog" + 0.050*"fox" + '
  '0.050*"ham"'),
 (5,
  '0.050*"sky" + 0.050*"blue" + 0.050*"beautiful" + 0.050*"lazy" + 0.050*"dog" '
  '+ 0.050*"quick" + 0.050*"brown" + 0.050*"f

## Compute the Coherence Scores

In [22]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=norm_corpus_tokens, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.38707259196030064


## Visualization

In [14]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.145651 -0.106416       1        1  36.661270
0     -0.004821 -0.035114       2        1  21.568339
7     -0.136495  0.131101       3        1  21.008319
9      0.210899  0.010563       4        1  13.617330
8      0.012678 -0.000024       5        1   1.190802
2      0.012678 -0.000023       6        1   1.190794
6      0.012678 -0.000023       7        1   1.190789
4      0.012678 -0.000022       8        1   1.190789
5      0.012679 -0.000021       9        1   1.190788
1      0.012677 -0.000021      10        1   1.190779, topic_info=         Term      Freq     Total Category  logprob  loglift
2         sky  4.000000  4.000000  Default  20.0000  20.0000
8        lazy  2.000000  2.000000  Default  19.0000  19.0000
9       quick  2.000000  2.000000  Default  18.0000  18.0000
6         fox  2.000000  2.000000  Default  17.0000  17.0000
4       brown  2.000000  2.000000  Default  16.0000  16.0000
..        ...       ...       ...      ...      ...      ...
5         dog  0.025603  2.840190  Topic10  -2.9957  -0.2784
6         fox  0.025602  2.840183  Topic10  -2.9957  -0.2784
0   beautiful  0.025605  3.203328  Topic10  -2.9956  -0.3986
1        blue  0.025608  4.130649  Topic10  -2.9955  -0.6527
2         sky  0.025605  4.493786  Topic10  -2.9956  -0.7371

[220 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
10        2  0.582256      bacon
10        4  0.582256      bacon
11        2  1.014680      beans
0         1  0.312175  beautiful
0         2  0.312175  beautiful
0         3  0.312175  beautiful
1         1  0.484185       blue
1         2  0.242093       blue
1         3  0.242093       blue
12        2  1.014682  breakfast
4         1  0.704179      brown
4         2  0.352089      brown
5         1  0.704178        dog
5         2  0.352089        dog
13        2  0.582256       eggs
13        4  0.582256       eggs
6         1  0.704180        fox
6         2  0.352090        fox
18        4  0.813463      green
14        2  0.582255        ham
14        4  0.582255        ham
7         2  1.014679      jumps
15        2  1.014680      kings
8         1  0.704181       lazy
8         2  0.352090       lazy
3         1  0.463686       love
3         4  0.463686       love
9         1  0.704179      quick
9         2  0.352089      quick
16        2  0.582256   sausages
16        4  0.582256   sausages
2         1  0.222530        sky
2         2  0.222530        sky
2         3  0.667589        sky
17        2  1.014680      toast
19        3  0.559325      today, R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 8, 10, 9, 3, 7, 5, 6, 2])

## Another Dataset

In [15]:
dset_url='https://archive.org/download/misc-dataset/airline-tweets-ngram.csv'
airline_df = pd.read_csv(dset_url) #Change this with the name of your downloaded file

airline_df['grams'] = airline_df['gram1_word_gt_1'].astype('string') + ','+airline_df['gram2_word_gt_1'].astype('string')+','+airline_df['gram3_word_gt_1'].astype('string')
# replace one or more commas with a single space character
airline_df['grams'] = airline_df['grams'].astype('string').replace(r'[,\s]+', r' ', regex=True) 
airline_df.dropna(inplace=True)

airline_grams =[s.split() for s in airline_df.grams.values.tolist()]

print(airline_grams[0:1])

[['seriously', 'would', 'pay', 'flight', 'seat', 'play', 'really', 'bad', 'thing', 'fly', 'va', 'would_pay', 'pay_flight', 'flight_seat', 'seat_not', 'not_play', 'really_bad', 'bad_thing', 'thing_fly', 'fly_va', 'bad_thing_fly']]


In [16]:
# create dictionary
airline_id2word = Dictionary(airline_grams)

# create corpus
airline_corpus = [airline_id2word.doc2bow(text) for text in airline_grams]

# test
print(airline_corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)]]


In [17]:
# test by checking the id and its corresponding word value and frequency
[[(airline_id2word[i], freq) for i, freq in doc] for doc in airline_corpus[0:1]]

[[('bad', 1),
  ('bad_thing', 1),
  ('bad_thing_fly', 1),
  ('flight', 1),
  ('flight_seat', 1),
  ('fly', 1),
  ('fly_va', 1),
  ('not_play', 1),
  ('pay', 1),
  ('pay_flight', 1),
  ('play', 1),
  ('really', 1),
  ('really_bad', 1),
  ('seat', 1),
  ('seat_not', 1),
  ('seriously', 1),
  ('thing', 1),
  ('thing_fly', 1),
  ('va', 1),
  ('would', 1),
  ('would_pay', 1)]]

In [18]:
# Build LDA model
airline_lda_model = LdaModel(corpus=airline_corpus,
                   id2word=airline_id2word,
                   num_topics=10, 
                   random_state=100,
                   update_every=1,
                   chunksize=100,
                   alpha='auto',
                   per_word_topics=True)

In [19]:
# Print the Keyword
pprint(airline_lda_model.print_topics())
doc_lda = airline_lda_model[airline_corpus]

[(0,
  '0.058*"call" + 0.034*"cannot" + 0.033*"get" + 0.025*"help" + 0.023*"flight" '
  '+ 0.023*"phone" + 0.021*"still" + 0.018*"make" + 0.018*"wait" + '
  '0.015*"call_back"'),
 (1,
  '0.166*"flight" + 0.077*"cancel" + 0.043*"flightled" + '
  '0.042*"cancel_flightled" + 0.025*"tomorrow" + 0.022*"cancel_flight" + '
  '0.019*"dfw" + 0.015*"need" + 0.014*"miss" + 0.014*"home"'),
 (2,
  '0.053*"get" + 0.044*"back" + 0.031*"flight_cancel" + 0.028*"work" + '
  '0.024*"us" + 0.024*"flight_cancel_flightled" + 0.022*"thank" + 0.022*"you" '
  '+ 0.019*"know" + 0.018*"let"'),
 (3,
  '0.070*"hold" + 0.037*"weather" + 0.030*"please" + 0.023*"email" + '
  '0.022*"hour" + 0.021*"response" + 0.015*"rebook" + 0.015*"info" + '
  '0.014*"do" + 0.013*"dm"'),
 (4,
  '0.035*"hour" + 0.028*"delay" + 0.027*"one" + 0.023*"time" + 0.021*"bad" + '
  '0.021*"even" + 0.016*"ever" + 0.015*"line" + 0.014*"would" + 0.014*"me"'),
 (5,
  '0.037*"gate" + 0.033*"change" + 0.027*"late" + 0.026*"agent" + '
  '0.023*"leav

In [20]:
# Compute Coherence Score
airline_coherence_model_lda = CoherenceModel(model=airline_lda_model, texts=airline_grams, dictionary=airline_id2word, coherence='c_v')
airline_coherence_lda = airline_coherence_model_lda.get_coherence()
print('\nCoherence Score: ', airline_coherence_lda)
# output:
# Coherence Score:  0.3016957987636324


Coherence Score:  0.3016957987636324


In [21]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(airline_lda_model, airline_corpus, airline_id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.183055  0.202947       1        1  17.007607
1      0.270859 -0.252698       2        1  14.419754
4     -0.172757 -0.128032       3        1  12.489746
6     -0.097420 -0.063627       4        1  10.663338
2     -0.003140  0.167719       5        1  10.005126
8     -0.085044 -0.046912       6        1   8.417625
5      0.053564  0.032261       7        1   7.529030
7     -0.025443  0.086639       8        1   6.694856
3     -0.100057 -0.000415       9        1   6.538820
9     -0.023618  0.002119      10        1   6.234097, topic_info=             Term         Freq        Total Category  logprob  loglift
3          flight  3425.000000  3425.000000  Default  30.0000  30.0000
375        cancel  1307.000000  1307.000000  Default  29.0000  29.0000
263          call  1160.000000  1160.000000  Default  28.0000  28.0000
215           get  1797.000000  1797.000000  Default  27.0000  27.0000
292      customer   744.000000   744.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
2889   ridiculous    45.955859    48.023921  Topic10  -5.0655   2.7311
283          need   181.864550   458.115131  Topic10  -3.6899   1.8513
106          help   250.534363   881.292179  Topic10  -3.3695   1.5173
288   reservation    56.636526   287.327346  Topic10  -4.8565   1.1512
399          hour    61.859833  1046.114266  Topic10  -4.7683  -0.0528

[433 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
173       7  0.976203          a
3626      4  0.997317         aa
1001      8  0.986197       able
3980      8  0.966653   able_get
1722      9  0.988233   actually
...     ...       ...        ...
1142      7  0.966382        wtf
33        8  0.993627        yes
640       1  0.987235  yesterday
75        7  0.994477        yet
133       5  0.995732        you

[508 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 5, 7, 3, 9, 6, 8, 4, 10])